In [1]:
from dotenv import load_dotenv
import os
import neptune
import praw
from datetime import datetime
import json
import time
import pandas as pd
from tqdm import tqdm

from pathlib import Path
Root = Path('.').absolute().parent
DATA = Root / r'C:\Users\Admin\Projects\ML Projects\ManipDetect\data'
# DATA = Root/ r'C:\Users\krishnadas\Projects\ML Projects\ManipDetect\data'

In [2]:
def reddit_connect():
    """Initialize Reddit connection"""
    load_dotenv()
    reddit = praw.Reddit(
        client_id=os.getenv('REDDIT_CLIENT_ID'),
        client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
        user_agent=os.getenv('REDDIT_USER_AGENT'),
        username=os.getenv('REDDIT_USERNAME'),
        password=os.getenv('REDDIT_PASSWORD')
    )
    return reddit

In [3]:
def load_update_progress(filename="author_update_progress.json"):
    """Load previous update progress"""
    try:
        with open(filename, 'r') as f:
            progress = json.load(f)
            return progress.get('completed_ids', set()), progress.get('failed_ids', set())
    except FileNotFoundError:
        return set(), set()

def save_update_progress(completed_ids, failed_ids, filename="author_update_progress.json"):
    """Save current update progress"""
    progress = {
        'completed_ids': list(completed_ids),
        'failed_ids': list(failed_ids),
        'last_updated': datetime.now().isoformat(),
        'total_completed': len(completed_ids),
        'total_failed': len(failed_ids)
    }
    with open(filename, 'w') as f:
        json.dump(progress, f, indent=2)

def analyze_csv_file(csv_file):
    """Analyze the CSV file to understand the data structure"""
    print(f"Analyzing {csv_file}...")
    
    df = pd.read_csv(csv_file)
    print(f"Total rows: {len(df):,}")
    print(f"Columns: {list(df.columns)}")
    
    # Check for post_id column
    if 'post_id' not in df.columns:
        print("ERROR: 'post_id' column not found!")
        print("Available columns:", list(df.columns))
        return None
    
    # Check for author columns
    author_columns = [col for col in df.columns if 'author' in col.lower()]
    print(f"Author-related columns: {author_columns}")
    
    # Analyze missing authors
    if author_columns:
        for col in author_columns:
            missing_count = df[col].isna().sum()
            empty_count = (df[col] == '').sum()
            deleted_count = (df[col] == '[deleted]').sum()
            unavailable_count = (df[col] == '[unavailable]').sum()
            
            print(f"\n{col} analysis:")
            print(f"  Missing (NaN): {missing_count:,}")
            print(f"  Empty strings: {empty_count:,}")
            print(f"  [deleted]: {deleted_count:,}")
            print(f"  [unavailable]: {unavailable_count:,}")
            
            total_to_update = missing_count + empty_count + unavailable_count
            print(f"  Total to potentially update: {total_to_update:,}")
    
    return df

def identify_posts_to_update(df, target_column='author_id'):
    """Identify which posts need author updates"""
    if target_column not in df.columns:
        print(f"Column '{target_column}' not found. Available columns: {list(df.columns)}")
        return []
    
    # Posts that need updating
    needs_update = df[
        df[target_column].isna() |
        (df[target_column] == '') |
        (df[target_column] == '[unavailable]')
    ]
    
    print(f"Posts needing author updates: {len(needs_update):,}")
    return needs_update['post_id'].tolist()

In [4]:
filepath = DATA / 'reddit_wsb.csv'
df = pd.read_csv(filepath)
df.head()

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56


In [5]:
# df['author_name'] = None
# df['author_id'] = None
# # chnage id column to post id
# df.rename(columns={'id': 'post_id'}, inplace=True)
# df.to_csv(DATA / 'new_reddit_wsb.csv', index=False)

In [7]:
def update_authors_bulk(csv_file, target_column='author_name', batch_size=100, test_mode=False, test_limit=100):
    """Update authors in bulk with progress tracking and resume capability"""
    
    # Load environment variables
    load_dotenv()
    
    # Initialize Neptune for tracking
    run = neptune.init_run(project="krishnadasm/wallstreetbets-scraper")
    run["config/task"] = "author_update"
    run["config/csv_file"] = csv_file
    run["config/target_column"] = target_column
    run["config/batch_size"] = batch_size
    run["config/test_mode"] = test_mode
    if test_mode:
        run["config/test_limit"] = test_limit
    
    # Initialize Reddit
    reddit = reddit_connect()
    
    # Load and analyze CSV
    df = analyze_csv_file(csv_file)
    if df is None:
        return
    
    # Identify posts to update
    posts_to_update = identify_posts_to_update(df, target_column)
    
    if not posts_to_update:
        print("No posts need updating!")
        run.stop()
        return
    
    # TEST MODE: Limit to first N posts
    if test_mode:
        posts_to_update = posts_to_update[:test_limit]
        print(f"🧪 TEST MODE: Limited to {len(posts_to_update)} posts")
    
    # Load previous progress (skip in test mode)
    if not test_mode:
        completed_ids, failed_ids = load_update_progress()
        remaining_posts = [pid for pid in posts_to_update if pid not in completed_ids]
    else:
        completed_ids, failed_ids = set(), set()
        remaining_posts = posts_to_update
    
    print(f"Total posts to update: {len(posts_to_update):,}")
    if not test_mode:
        print(f"Already completed: {len(completed_ids):,}")
    print(f"Remaining: {len(remaining_posts):,}")
    
    # Log initial stats to Neptune
    run["stats/total_posts_to_update"] = len(posts_to_update)
    run["stats/already_completed"] = len(completed_ids)
    run["stats/remaining"] = len(remaining_posts)
    
    # Estimate time
    if test_mode:
        estimated_minutes = len(remaining_posts) / 60  # ~1 post per second with delays
        print(f"Estimated time: {estimated_minutes:.1f} minutes")
    else:
        estimated_hours = len(remaining_posts) / 3600  # ~1 post per second with delays
        print(f"Estimated time: {estimated_hours:.1f} hours")
    
    # Start updating
    start_time = time.time()
    success_count = 0
    error_count = 0
    
    try:
        # Process in batches
        for i in range(0, len(remaining_posts), batch_size):
            batch = remaining_posts[i:i+batch_size]
            
            print(f"\nProcessing batch {i//batch_size + 1}/{(len(remaining_posts) + batch_size - 1)//batch_size}")
            
            for post_id in tqdm(batch, desc="Updating authors"):
                try:
                    # Get submission
                    submission = reddit.submission(id=post_id)
                    
                    # Get author info (both name and ID)
                    if submission.author is not None:
                        author_name = submission.author.name
                        author_id = submission.author.id
                    else:
                        author_name = "[deleted]"
                        author_id = "[deleted]"
                    
                    # Update dataframe
                    mask = df['post_id'] == post_id
                    df.loc[mask, target_column] = author_name
                    
                    # Add author_id column if it doesn't exist
                    if 'author_id' not in df.columns:
                        df['author_id'] = None
                    df.loc[mask, 'author_id'] = author_id
                    
                    completed_ids.add(post_id)
                    success_count += 1
                    
                    # In test mode, show detailed results
                    if test_mode and success_count <= 10:
                        print(f"  ✅ {post_id}: {author_name} (ID: {author_id})")
                    
                except Exception as e:
                    print(f"  ❌ Error updating post {post_id}: {e}")
                    failed_ids.add(post_id)
                    error_count += 1
                
                # Rate limiting (shorter delay in test mode)
                if test_mode:
                    time.sleep(0.5)  # Faster for testing
                else:
                    time.sleep(1.1)  # Conservative for production
            
            # Save progress after each batch (skip in test mode)
            if not test_mode:
                save_update_progress(completed_ids, failed_ids)
            
            # Save intermediate CSV
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            if test_mode:
                temp_filename = f"test_updated_{timestamp}.csv"
            else:
                temp_filename = f"temp_updated_{timestamp}.csv"
            df.to_csv(temp_filename, index=False)
            
            # Log progress to Neptune
            run["progress/completed"].log(len(completed_ids))
            run["progress/failed"].log(len(failed_ids))
            run["progress/success_rate"].log(success_count / (success_count + error_count) if (success_count + error_count) > 0 else 0)
            
            elapsed_time = time.time() - start_time
            posts_per_minute = len(completed_ids) / (elapsed_time / 60) if elapsed_time > 0 else 0
            run["progress/posts_per_minute"].log(posts_per_minute)
            
            print(f"Progress: {len(completed_ids):,}/{len(posts_to_update):,} ({len(completed_ids)/len(posts_to_update)*100:.1f}%)")
            print(f"Success rate: {success_count}/{success_count + error_count} ({success_count/(success_count + error_count)*100:.1f}%)")
            print(f"Posts per minute: {posts_per_minute:.1f}")
            
            # Shorter break in test mode
            if i + batch_size < len(remaining_posts):
                if test_mode:
                    print("Taking a 5-second break...")
                    time.sleep(5)
                else:
                    print("Taking a 30-second break between batches...")
                    time.sleep(30)
    
    except KeyboardInterrupt:
        print("\nUpdate interrupted by user. Progress saved.")
        run["status"] = "interrupted"
        run["interruption_point"] = len(completed_ids)
    
    except Exception as e:
        print(f"Unexpected error: {e}")
        run["status"] = "error"
        run["error_message"] = str(e)
    
    finally:
        # Final save
        if not test_mode:
            save_update_progress(completed_ids, failed_ids)
        
        # Save final CSV
        if test_mode:
            final_filename = Path(str(csv_file).replace(".csv", "_test_with_authors.csv"))
        else:
            final_filename = Path(str(csv_file).replace('.csv', '_with_authors.csv'))
        df.to_csv(final_filename, index=False)
        
        # Calculate final stats
        end_time = time.time()
        total_time = end_time - start_time
        
        print(f"\nUpdate completed!")
        if test_mode:
            print(f"Total time: {total_time/60:.2f} minutes")
        else:
            print(f"Total time: {total_time/3600:.2f} hours")
        print(f"Successfully updated: {success_count:,}")
        print(f"Failed: {error_count:,}")
        print(f"Final CSV saved as: {final_filename}")
        
        # Show sample of updated data in test mode
        if test_mode and success_count > 0:
            print(f"\n📊 Sample of updated data:")
            sample_data = df[df['post_id'].isin(list(completed_ids)[:5])]
            for _, row in sample_data.iterrows():
                print(f"  {row['post_id']}: '{row['title'][:50]}...' by {row[target_column]} (ID: {row['author_id']})")
        
        # Log final results to Neptune
        run["results/total_time_hours"] = total_time / 3600
        run["results/successfully_updated"] = success_count
        run["results/failed_updates"] = error_count
        run["results/final_csv"] = str(final_filename)
        
        # Upload final CSV to Neptune
        run["data/updated_csv"].upload(str(final_filename))
        if not test_mode:
            run["data/progress_file"].upload("author_update_progress.json")
        
        run.stop()
        
        return df

def resume_update(csv_file, target_column='author_name'):
    """Resume a previously interrupted update"""
    print("Resuming previous update...")
    return update_authors_bulk(csv_file, target_column)

def quick_stats(csv_file):
    """Get quick statistics about the CSV file"""
    df = analyze_csv_file(csv_file)
    if df is not None:
        posts_to_update = identify_posts_to_update(df)
        print(f"\nQuick Stats:")
        print(f"Total posts: {len(df):,}")
        print(f"Posts needing author updates: {len(posts_to_update):,}")
        print(f"Estimated time: {len(posts_to_update)/3600:.1f} hours")

def test_author_update(csv_file, target_column='author_name', test_limit=100):
    """Test author update with a small sample first"""
    print(f"🧪 TESTING MODE: Updating {test_limit} posts")
    print("=" * 50)
    return update_authors_bulk(csv_file, target_column, test_mode=True, test_limit=test_limit)

def production_author_update(csv_file, target_column='author_name'):
    """Run full production update"""
    print("🚀 PRODUCTION MODE: Updating all missing authors")
    print("=" * 50)
    return update_authors_bulk(csv_file, target_column, test_mode=False)


In [ ]:

if __name__ == "__main__":
    # Configuration
    CSV_FILE = DATA/"new_reddit_wsb.csv"  # Replace with your actual CSV file
    TARGET_COLUMN = "author_name"  # Column to update
    
    print("🔧 WSB Author Updater Script")
    print("=" * 50)
    
    # Step 1: Get quick statistics first
    print("Step 1: Analyzing your CSV file...")
    quick_stats(CSV_FILE)
    
    print("\n" + "=" * 50)
    print("Choose your action:")
    print("1. Test Mode (100 posts) - Uncomment line below")
    print("2. Production Mode (all posts) - Uncomment line below")
    print("3. Resume interrupted update - Uncomment line below")
    print("=" * 50)
    
    # Uncomment ONE of these lines to run:
    
    # 🧪 TEST MODE - Start here to test with 100 posts
    # test_author_update(CSV_FILE, TARGET_COLUMN, test_limit=100)
    
    # 🚀 PRODUCTION MODE - Run this after testing works
    production_author_update(CSV_FILE, TARGET_COLUMN)
    
    # 🔄 RESUME MODE - Use this if production run was interrupted
    # resume_update(CSV_FILE, TARGET_COLUMN)

🔧 WSB Author Updater Script
Step 1: Analyzing your CSV file...
Analyzing C:\Users\Admin\Projects\ML Projects\ManipDetect\data\new_reddit_wsb.csv...
Total rows: 53,187
Columns: ['title', 'score', 'post_id', 'url', 'comms_num', 'created', 'body', 'timestamp', 'author_name', 'author_id']
Author-related columns: ['author_name', 'author_id']

author_name analysis:
  Missing (NaN): 53,187
  Empty strings: 0
  [deleted]: 0
  [unavailable]: 0
  Total to potentially update: 53,187

author_id analysis:
  Missing (NaN): 53,187
  Empty strings: 0
  [deleted]: 0
  [unavailable]: 0
  Total to potentially update: 53,187
Posts needing author updates: 53,187

Quick Stats:
Total posts: 53,187
Posts needing author updates: 53,187
Estimated time: 14.8 hours

Choose your action:
1. Test Mode (100 posts) - Uncomment line below
2. Production Mode (all posts) - Uncomment line below
3. Resume interrupted update - Uncomment line below
🚀 PRODUCTION MODE: Updating all missing authors
[neptune] [info   ] Neptune i

Updating authors:   0%|          | 0/100 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_8808\1706575410.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[deleted]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, target_column] = author_name
C:\Users\Admin\AppData\Local\Temp\ipykernel_8808\1706575410.py:96: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[deleted]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'author_id'] = author_id
Updating authors:  16%|█▌        | 16/100 [00:41<02:48,  2.01s/it]

  ❌ Error updating post l6u5j2: 'Redditor' object has no attribute 'id'


Updating authors:  33%|███▎      | 33/100 [01:26<02:44,  2.46s/it]

  ❌ Error updating post l6tke3: 'Redditor' object has no attribute 'id'


...

  ❌ Error updating post l6wyap: 'Redditor' object has no attribute 'id'


Updating authors:  55%|█████▌    | 55/100 [01:28<01:10,  1.57s/it]

  ❌ Error updating post l6wxyd: 'Redditor' object has no attribute 'id'


Updating authors: 100%|██████████| 100/100 [02:40<00:00,  1.60s/it]


Progress: 775/53,187 (1.5%)
Success rate: 775/800 (96.9%)
Posts per minute: 26.4
Taking a 30-second break between batches...


In [10]:
filepath = DATA/"new_reddit_wsb_test_with_authors.csv"
df = pd.read_csv(filepath)
df.head()


,title,score,post_id,url,comms_num,created,body,timestamp,author_name,author_id
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41,[deleted],[deleted]
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10,jaxxtothemaxx,onvag
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,stonerbobo,f3p9m
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57,Sleavitt10,9u7y1
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56,di3_b0ld,13cexg
